In [25]:
%matplotlib inline

import collections
import itertools
import json

import matplotlib.pyplot as plt
import pandas as pd

In [26]:
with open('data/cards.json') as f:
    data = json.load(f)
    
# This yields an iterator that returns one card at a time.
cards = itertools.chain.from_iterable(data.values())
series = [pd.Series(card) for card in cards]
cards = pd.concat(series, axis=1, join='outer').T.set_index('cardId')
cards.head()

,artist,attack,cardSet,classes,collectible,cost,durability,elite,faction,flavor,...,imgGold,locale,mechanics,multiClassGroup,name,playerClass,race,rarity,text,type
cardId,,,,,,,,,,,,,,,,,,,,,
CRED_15,NaN,1,Credits,NaN,NaN,1,NaN,True,NaN,NaN,...,http://wow.zamimg.com/images/hearthstone/cards...,enUS,NaN,NaN,Andy Brock,Neutral,NaN,Legendary,"Can't be <b>Silenced. Divine Shield, Stealth.</b>",Minion
CRED_21,NaN,1,Credits,NaN,NaN,1,NaN,True,NaN,NaN,...,http://wow.zamimg.com/images/hearthstone/cards...,enUS,NaN,NaN,Bryan Chang,Neutral,NaN,Legendary,<b>Foodie:</b> Make all minions edible.,Minion
CRED_06,NaN,3,Credits,NaN,NaN,1,NaN,True,NaN,NaN,...,http://wow.zamimg.com/images/hearthstone/cards...,enUS,NaN,NaN,Derek Sakamoto,Neutral,NaN,Legendary,<i>The notorious Footclapper.</i>,Minion
CRED_04,NaN,3,Credits,NaN,NaN,1,NaN,True,NaN,NaN,...,http://wow.zamimg.com/images/hearthstone/cards...,enUS,NaN,NaN,Steven Gabriel,Neutral,NaN,Legendary,<b>Battlecry:</b> Summon a frothy beverage.,Minion
CRED_18,NaN,2,Credits,NaN,NaN,2,NaN,True,NaN,NaN,...,http://wow.zamimg.com/images/hearthstone/cards...,enUS,NaN,NaN,Becca Abel,Neutral,NaN,Legendary,"Whenever you draw a card, make it Golden.",Minion


In [31]:
with open('data/info.json') as f:
    metadata = json.load(f)

sorted(metadata)

['classes',
 'factions',
 'locales',
 'patch',
 'qualities',
 'races',
 'sets',
 'standard',
 'types',
 'wild']